#### Initialize

In [1]:
import logging, os, struct, urllib
from sqlalchemy import create_engine, MetaData, Table
from azure.monitor.query import LogsQueryClient
from azure.identity import DefaultAzureCredential
from datetime import timedelta, datetime, timezone
from module.logsanalytics import LogsAnalytics
from module.warehouse import Warehouse
import pandas as pd
def_credential = DefaultAzureCredential()

logs_id = os.environ["LOGS_ANALYTICS_ID"]
lg = LogsAnalytics(logs_id=logs_id, credential=def_credential)

warehouse = Warehouse(
    server=os.environ["DB_SERVER"],
    database=os.environ["DB_NAME"],
    credential=def_credential
)

#### Run Jobs

In [5]:
# ## Define the save jobs
# save_list = [
#     ## log_message, table_name, function_name
#     ('TIMER_UPDATE_LOGS: refresh ad_audits logs',           'ad_audits',         'list_audits'),
#     ('TIMER_UPDATE_LOGS: refresh ad_signins logs',          'ad_signins',        'list_signins')
# ]

# ## run all save jobs
# for job in save_list:


#     ## timeframe from last update till now
#     logging_message = job[0]
#     table_name = job[1]
#     list_func = getattr(lg, job[2])

#     logging.info(logging_message)


#     ## retrieve new rows
#     list_from = warehouse.get_max(table_name,'TimeGenerated') + timedelta(milliseconds=1)
#     list_to   = datetime.now(tz=timezone.utc)
#     df = list_func(time_span=(list_from, list_to))

#     ## save to warehouse
#     warehouse.append(table_name, df)


#### Test - Audits

In [3]:
ts=timedelta(days=7)
df = lg.list_audits_alerts(time_span=ts)
df.columns

Index(['TimeGenerated', 'Category', 'AADOperationType', 'ActivityDisplayName',
       'Result', 'initiatedby_userid', 'initiatedby_upn', 'initiatedby_name',
       'initiatedby_ip', 'target_id', 'target_displayName', 'target_upn',
       'target_type', 'additional_detail', 'modified_properties'],
      dtype='object')

#### Test - Signins

In [26]:
query = r'''
SigninLogs
| extend FailureOrSuccess = iff(ResultType in ("0", "50125", "50140"), "Success", "Failure")
| extend Country=tostring(LocationDetails.countryOrRegion)
| extend City=tostring(LocationDetails.city)
| extend DeviceId=tostring(DeviceDetail.deviceId)
| extend DeviceDisplayName=tostring(DeviceDetail.displayName)
| extend DeviceOS=tostring(DeviceDetail.operatingSystem)
| extend DeviceTrustType=tostring(DeviceDetail.trustType)
| extend DeviceBrowser=tostring(DeviceDetail.browser)
| extend StatusErrorCode=tostring(Status.errorCode)
| extend StatusAdditionalDetails=tostring(Status.additionalDetails)
| project OperationName, Category, ResultType, UserDisplayName, UserPrincipalName, UserType, Country, City, IPAddress, AppDisplayName, ClientAppUsed, ResourceDisplayName, DeviceId, DeviceDisplayName, DeviceOS, DeviceTrustType, DeviceBrowser, StatusErrorCode, StatusAdditionalDetails
'''
ts=timedelta(days=7)
df = lg.list(query, time_span=ts)
df.columns

Index(['OperationName', 'Category', 'ResultType', 'UserDisplayName',
       'UserPrincipalName', 'UserType', 'Country', 'City', 'IPAddress',
       'AppDisplayName', 'ClientAppUsed', 'ResourceDisplayName', 'DeviceId',
       'DeviceDisplayName', 'DeviceOS', 'DeviceTrustType', 'DeviceBrowser',
       'StatusErrorCode', 'StatusAdditionalDetails'],
      dtype='object')

In [40]:
good = df.StatusErrorCode == '0'
good_df = df[good]
gb = good_df.groupby(['Country','IPAddress'])
# df2= gb.count()
df2=gb.agg({
    'UserPrincipalName': 'nunique',
    'DeviceId': 'nunique',
})

In [43]:
gb = good_df.groupby(['Country'])
gb['DeviceId'].nunique()

Country
AE     2
AR     1
CN     3
HK     1
ID    43
IN     1
JP     1
KH     1
MA     9
MM     1
MY    78
NG     1
NO     7
PH    37
PK    27
SG    98
TH    15
TW     1
US     1
Name: DeviceId, dtype: int64

In [45]:
audits_df = lg.list_audits()